## Backtesting 


Backtesting is one of the most important stages when you start trading or investing. So what is backtesting?

Backtesting is the general method for seeing how well a strategy or model would have done ex-post. Backtesting assesses the viability of a trading strategy by discovering how it would play out using historical data. If backtesting works, traders and analysts may have the confidence to employ it going forward. [Investopedia](https://www.investopedia.com/terms/b/backtesting.asp)

When you backtest your stock (or portfolio), you have to define your strategy. Backtesting can be performed in many ways. It can include multiple indicators such as SMA, RSI, Bollinger band, etc. Or using statistical such as variance, drawdown, etc.  However, put these strategy from ideas to codes is a huge problem, it will consume lots of time and effort and even knowledge to build a module for this process. In order to reduce time, I will use [backtesting.py](https://kernc.github.io/backtesting.py) library to perform this task.

In [2]:
import vnquant.DataLoader as dl
import pandas as pd
import numpy as np
import datetime
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.lib import resample_apply
from backtesting.test import SMA

/Users/haquochung/opt/anaconda3/lib/python3.8/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [3]:
#stock data
def load_stock_data(symbol):
    start = '2020-01-01'
    
    #you can fix the time frame by using timedelta function of datetime library
    now = datetime.datetime.now()
    end = now.strftime("%Y-%m-%d")
    loader = dl.DataLoader(symbol, start,end, data_source='VND', minimal=True)
    pricedata = loader.download()
    
    #format the data for the mplfinance
    stock = pricedata.copy()
    stock.reset_index(inplace = True)

    dailyInfo = pd.DataFrame( columns =['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
    dailyInfo['Date'] = stock['date']
    dailyInfo['Open'] = stock['open']
    dailyInfo['High'] = stock['high']
    dailyInfo['Low'] = stock['low']
    dailyInfo['Close'] = stock['close']
    dailyInfo['Volume'] = stock['volume']
    dailyInfo.set_index('Date', inplace= True)
    
    
    
    #to store data remove the '#' on the following line
    #csv_file = close_data.to_csv(f'Data/ClosePrice/{symbol} historical since {start}', index=True)

    return dailyInfo

In [4]:
#Simple backtesting using backtesting.py
#simple sma backtest
symbol = input('Please enter a symbol: ')
symbol = symbol.upper()
stock = load_stock_data(symbol)


class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()


bt = Backtest(stock, SmaCross,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run()
bt.plot()

Please enter a symbol: TCB


2022-02-25 08:53:13,829 : INFO : NumExpr defaulting to 8 threads.
2022-02-25 08:53:13,837 : INFO : data TCB from 2020-01-01 to 2022-02-25 have already cloned!


Row(id='1434', ...)

In [5]:
output

Start                     2020-01-02 00:00:00
End                       2022-02-25 00:00:00
Duration                    785 days 00:00:00
Exposure Time [%]                   94.402985
Equity Final [$]                    2752.5556
Equity Peak [$]                       10000.0
Return [%]                         -72.474444
Buy & Hold Return [%]              112.184874
Return (Ann.) [%]                  -45.475418
Volatility (Ann.) [%]               20.220838
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -72.474444
Avg. Drawdown [%]                  -72.474444
Max. Drawdown Duration      737 days 00:00:00
Avg. Drawdown Duration      737 days 00:00:00
# Trades                                   35
Win Rate [%]                        25.714286
Best Trade [%]                      35.898416
Worst Trade [%]                    -26.307722
Avg. Trade [%]                    

In [6]:
#backtesting sma and rsi

def SMA(array, n):
    """Simple moving average"""
    return pd.Series(array).rolling(n).mean()


def RSI(array, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

class System(Strategy):
    d_rsi = 30  # Daily RSI lookback periods
    w_rsi = 30  # Weekly
    level = 70
    
    def init(self):
        # Compute moving averages the strategy demands
        self.ma10 = self.I(SMA, self.data.Close, 10)
        self.ma20 = self.I(SMA, self.data.Close, 20)
        self.ma50 = self.I(SMA, self.data.Close, 50)
        self.ma100 = self.I(SMA, self.data.Close, 100)
        
        # Compute daily RSI(30)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)
        
        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', RSI, self.data.Close, self.w_rsi)
        
        
    def next(self):
        price = self.data.Close[-1]
        
        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
            self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):
            
            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)
        
        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()


backtest = Backtest(stock, System, commission=.002)
backtest.run()



backtest.optimize(d_rsi=range(10, 35, 5),
                  w_rsi=range(10, 35, 5),
                  level=range(30, 80, 10))

backtest.plot()
#print(backtest)

/Users/haquochung/opt/anaconda3/lib/python3.8/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/9 [00:00<?, ?it/s]

Row(id='2323', ...)